In [1]:
cd ../astropixel/

/home/savannahgramze/codeastro/astropixel/astropixel


/home/savannahgramze/anaconda3/envs/codeastro/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
import catalog_querry
import cutout
import plot_stars
import make_star
